In [45]:
import pandas as pd
import requests
import json
import geopandas as gp
import numpy as np

In [2]:
crimenes = pd.read_csv("../Municipal-Delitos-2015-2020_jul2020/Municipal-Delitos-2015-2020_jul2020.csv", encoding="latin-1")

In [3]:
crimenes["Tipo de delito"].value_counts()

Robo                                                                             494676
Homicidio                                                                        123669
Lesiones                                                                         123669
Secuestro                                                                         68705
Feminicidio                                                                       54964
Otros delitos que atentan contra la libertad personal                             13741
Otros delitos que atentan contra la vida y la integridad corporal                 13741
Violencia de género en todas sus modalidades distinta a la violencia familiar     13741
Otros delitos que atentan contra la libertad y la seguridad sexual                13741
Violencia familiar                                                                13741
Aborto                                                                            13741
Narcomenudeo                    

In [6]:
crimes = ["Robo", "Homicidio", "Lesiones","Secuestro", "Feminicidio"]

In [8]:
specific_crimes = crimenes.loc[crimenes["Tipo de delito"].isin(crimes)]

In [10]:
specific_crimes[["Clave_Ent", "Cve. Municipio", "Año"]] = specific_crimes[["Clave_Ent", "Cve. Municipio", "Año"]].astype(str)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
specific_crimes["Total"] = specific_crimes.sum(axis=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
crimenes_period = specific_crimes[['Año','Clave_Ent','Cve. Municipio','Entidad','Municipio','Total']]

In [13]:
crimenes_period.head()

,Año,Clave_Ent,Cve. Municipio,Entidad,Municipio,Total
0,2015,1,1001,Aguascalientes,Aguascalientes,10.0
1,2015,1,1001,Aguascalientes,Aguascalientes,4.0
2,2015,1,1001,Aguascalientes,Aguascalientes,10.0
3,2015,1,1001,Aguascalientes,Aguascalientes,2.0
4,2015,1,1001,Aguascalientes,Aguascalientes,1.0


In [15]:
final_df = crimenes_period.groupby(['Año', 'Clave_Ent', 'Cve. Municipio', 'Entidad', 'Municipio']).sum().reset_index()

In [17]:
len(final_df["Municipio"].unique())

2326

In [20]:
crimes_2019_df = final_df.loc[final_df["Año"]=="2019"]

In [21]:
crimes_2019_df.head()

,Año,Clave_Ent,Cve. Municipio,Entidad,Municipio,Total
8799,2019,1,1001,Aguascalientes,Aguascalientes,12977.0
8800,2019,1,1002,Aguascalientes,Asientos,421.0
8801,2019,1,1003,Aguascalientes,Calvillo,397.0
8802,2019,1,1004,Aguascalientes,Cosío,157.0
8803,2019,1,1005,Aguascalientes,Jesús María,1687.0


## Population

In [18]:
municipios1 = pd.read_csv("../base_municipios_final_datos_01.csv", encoding="latin-1")
municipios2 = pd.read_csv("../base_municipios_final_datos_02.csv", encoding="latin-1")
municipios = pd.concat([municipios1, municipios2])
municipios.drop("RENGLON", axis=1, inplace=True)
municipios[["CLAVE", "CLAVE_ENT", "AÑO"]] = municipios[["CLAVE", "CLAVE_ENT", "AÑO"]].astype(str)

In [22]:
muncipios_2019 = municipios.loc[municipios["AÑO"]=="2019"]

In [23]:
ages = ["pobm_00_04", "pobm_05_09", "pobm_10_14", "pobm_15_19"]

In [24]:
ranges_remove = muncipios_2019.loc[muncipios_2019["EDAD_QUIN"].isin(ages)].index

In [25]:
cirmes_2019 = muncipios_2019.drop(ranges_remove, axis=0)

In [27]:
rate_crimes = cirmes_2019.drop("EDAD_QUIN", axis=1)
rate_crimes.drop("SEXO", axis=1, inplace=True)
rate_crimes["NOM_ENT"].replace(['Oaxaca-Región Sierra Norte',
                       'Oaxaca-Región Papaloapam', 'Oaxaca-Región Mixteca',
       'Oaxaca-Región Istmo', 'Oaxaca-Región Valles Centrales',
       'Oaxaca-Región Sierra Sur', 'Oaxaca-Región Costa',
       'Oaxaca-Región Cañada'], "Oaxaca", inplace=True)
rate_crimes_2019 = rate_crimes.groupby(["CLAVE", "CLAVE_ENT", "NOM_ENT", "MUN"]).sum().reset_index()

In [67]:
rate_crimes_2019

,CLAVE,CLAVE_ENT,NOM_ENT,MUN,POB
0,10001,10,Durango,Canatlán,17076
1,10002,10,Durango,Canelas,1922
2,10003,10,Durango,Coneto de Comonfort,2063
3,10004,10,Durango,Cuencamé,17445
4,10005,10,Durango,Durango,316159
...,...,...,...,...,...
2452,9013,9,Ciudad de México,Xochimilco,225339
2453,9014,9,Ciudad de México,Benito Juárez,295292
2454,9015,9,Ciudad de México,Cuauhtémoc,338215
2455,9016,9,Ciudad de México,Miguel Hidalgo,241854


In [30]:
merged_crimes = pd.merge(crimes_2019_df, rate_crimes_2019, left_on=["Clave_Ent","Cve. Municipio"], right_on=["CLAVE_ENT", "CLAVE"])

In [31]:
merged_crimes

,Año,Clave_Ent,Cve. Municipio,Entidad,Municipio,Total,CLAVE,CLAVE_ENT,NOM_ENT,MUN,POB
0,2019,1,1001,Aguascalientes,Aguascalientes,12977.0,1001,1,Aguascalientes,Aguascalientes,446402
1,2019,1,1002,Aguascalientes,Asientos,421.0,1002,1,Aguascalientes,Asientos,20606
2,2019,1,1003,Aguascalientes,Calvillo,397.0,1003,1,Aguascalientes,Calvillo,27404
3,2019,1,1004,Aguascalientes,Cosío,157.0,1004,1,Aguascalientes,Cosío,7060
4,2019,1,1005,Aguascalientes,Jesús María,1687.0,1005,1,Aguascalientes,Jesús María,52424
...,...,...,...,...,...,...,...,...,...,...,...
2452,2019,9,9013,Ciudad de México,Xochimilco,3444.0,9013,9,Ciudad de México,Xochimilco,225339
2453,2019,9,9014,Ciudad de México,Benito Juárez,11305.0,9014,9,Ciudad de México,Benito Juárez,295292
2454,2019,9,9015,Ciudad de México,Cuauhtémoc,17482.0,9015,9,Ciudad de México,Cuauhtémoc,338215
2455,2019,9,9016,Ciudad de México,Miguel Hidalgo,9345.0,9016,9,Ciudad de México,Miguel Hidalgo,241854


In [32]:
merged_2 = merged_crimes.drop(["CLAVE", "CLAVE_ENT", "NOM_ENT", "MUN"], axis=1)

In [33]:
merged_2["Crime_Rate"] = (merged_2["Total"]/merged_2["POB"])*100000

In [35]:
merged_2

,Año,Clave_Ent,Cve. Municipio,Entidad,Municipio,Total,POB,Crime_Rate
0,2019,1,1001,Aguascalientes,Aguascalientes,12977.0,446402,2907.021026
1,2019,1,1002,Aguascalientes,Asientos,421.0,20606,2043.094244
2,2019,1,1003,Aguascalientes,Calvillo,397.0,27404,1448.693621
3,2019,1,1004,Aguascalientes,Cosío,157.0,7060,2223.796034
4,2019,1,1005,Aguascalientes,Jesús María,1687.0,52424,3217.991759
...,...,...,...,...,...,...,...,...
2452,2019,9,9013,Ciudad de México,Xochimilco,3444.0,225339,1528.363932
2453,2019,9,9014,Ciudad de México,Benito Juárez,11305.0,295292,3828.413909
2454,2019,9,9015,Ciudad de México,Cuauhtémoc,17482.0,338215,5168.901438
2455,2019,9,9016,Ciudad de México,Miguel Hidalgo,9345.0,241854,3863.901362


## All ages

In [49]:
area = pd.read_excel("../DENSIDAD-POB-MUNS-MEXICO.xlsx")

In [50]:
area

,NOM_MUN,CVMUN,POBTOT15,AREAKM
0,Aguascalientes,1001,877190.0,1178.10
1,Asientos,1002,46464.0,549.01
2,Calvillo,1003,56048.0,932.74
3,Cosío,1004,15577.0,129.74
4,Jesús María,1005,120405.0,504.99
...,...,...,...,...
2453,Villa Hidalgo,32054,19155.0,375.36
2454,Villanueva,32055,30240.0,2179.16
2455,Zacatecas,32056,146147.0,441.50
2456,Trancoso,32057,19413.0,220.81


In [51]:
area.drop("POBTOT15", axis=1, inplace=True)

In [70]:
area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   NOM_MUN  2458 non-null   object 
 1   CVMUN    2458 non-null   int64  
 2   AREAKM   2458 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 57.7+ KB


In [71]:
area["CVMUN"] = area["CVMUN"].astype(str)

In [75]:
muncipios_2019

,CLAVE,CLAVE_ENT,NOM_ENT,MUN,SEXO,AÑO,EDAD_QUIN,POB
4,1001,1,Aguascalientes,Aguascalientes,Mujeres,2019,pobm_00_04,38272
20,1002,1,Aguascalientes,Asientos,Mujeres,2019,pobm_00_04,2522
36,1003,1,Aguascalientes,Calvillo,Mujeres,2019,pobm_00_04,2738
52,1004,1,Aguascalientes,Cosío,Mujeres,2019,pobm_00_04,803
68,1005,1,Aguascalientes,Jesús María,Mujeres,2019,pobm_00_04,6359
...,...,...,...,...,...,...,...,...
653556,32054,32,Zacatecas,Villa Hidalgo,Hombres,2019,pobm_65_mm,672
653572,32055,32,Zacatecas,Villanueva,Hombres,2019,pobm_65_mm,1816
653588,32056,32,Zacatecas,Zacatecas,Hombres,2019,pobm_65_mm,3944
653604,32057,32,Zacatecas,Trancoso,Hombres,2019,pobm_65_mm,489


In [76]:
population_2019 = muncipios_2019.drop("EDAD_QUIN", axis=1)
population_2019.drop("SEXO", axis=1, inplace=True)
population_2019["NOM_ENT"].replace(['Oaxaca-Región Sierra Norte',
                       'Oaxaca-Región Papaloapam', 'Oaxaca-Región Mixteca',
       'Oaxaca-Región Istmo', 'Oaxaca-Región Valles Centrales',
       'Oaxaca-Región Sierra Sur', 'Oaxaca-Región Costa',
       'Oaxaca-Región Cañada'], "Oaxaca", inplace=True)
population_2019_final = population_2019.groupby(["CLAVE", "CLAVE_ENT", "NOM_ENT", "MUN"]).sum().reset_index()

In [77]:
population_2019_final

,CLAVE,CLAVE_ENT,NOM_ENT,MUN,POB
0,10001,10,Durango,Canatlán,34485
1,10002,10,Durango,Canelas,4782
2,10003,10,Durango,Coneto de Comonfort,4630
3,10004,10,Durango,Cuencamé,37200
4,10005,10,Durango,Durango,688253
...,...,...,...,...,...
2452,9013,9,Ciudad de México,Xochimilco,419426
2453,9014,9,Ciudad de México,Benito Juárez,430354
2454,9015,9,Ciudad de México,Cuauhtémoc,547703
2455,9016,9,Ciudad de México,Miguel Hidalgo,378708


In [78]:
with_km = pd.merge(population_2019_final, area, left_on="CLAVE", right_on="CVMUN")

In [79]:
with_km

,CLAVE,CLAVE_ENT,NOM_ENT,MUN,POB,NOM_MUN,CVMUN,AREAKM
0,10001,10,Durango,Canatlán,34485,Canatlán,10001,3491.96
1,10002,10,Durango,Canelas,4782,Canelas,10002,892.56
2,10003,10,Durango,Coneto de Comonfort,4630,Coneto de Comonfort,10003,1074.47
3,10004,10,Durango,Cuencamé,37200,Cuencamé,10004,5349.36
4,10005,10,Durango,Durango,688253,Durango,10005,9285.39
...,...,...,...,...,...,...,...,...
2452,9013,9,Ciudad de México,Xochimilco,419426,Xochimilco,9013,114.12
2453,9014,9,Ciudad de México,Benito Juárez,430354,Benito Juárez,9014,26.70
2454,9015,9,Ciudad de México,Cuauhtémoc,547703,Cuauhtémoc,9015,32.52
2455,9016,9,Ciudad de México,Miguel Hidalgo,378708,Miguel Hidalgo,9016,46.39


In [80]:
with_km["hab/km2"] = with_km["POB"]/with_km["AREAKM"]

In [81]:
with_km

,CLAVE,CLAVE_ENT,NOM_ENT,MUN,POB,NOM_MUN,CVMUN,AREAKM,hab/km2
0,10001,10,Durango,Canatlán,34485,Canatlán,10001,3491.96,9.875543
1,10002,10,Durango,Canelas,4782,Canelas,10002,892.56,5.357623
2,10003,10,Durango,Coneto de Comonfort,4630,Coneto de Comonfort,10003,1074.47,4.309101
3,10004,10,Durango,Cuencamé,37200,Cuencamé,10004,5349.36,6.954103
4,10005,10,Durango,Durango,688253,Durango,10005,9285.39,74.122142
...,...,...,...,...,...,...,...,...,...
2452,9013,9,Ciudad de México,Xochimilco,419426,Xochimilco,9013,114.12,3675.306695
2453,9014,9,Ciudad de México,Benito Juárez,430354,Benito Juárez,9014,26.70,16118.127341
2454,9015,9,Ciudad de México,Cuauhtémoc,547703,Cuauhtémoc,9015,32.52,16842.035670
2455,9016,9,Ciudad de México,Miguel Hidalgo,378708,Miguel Hidalgo,9016,46.39,8163.569735


In [83]:
info_country = pd.merge(with_km, merged_2, left_on=["CLAVE_ENT","CLAVE"],right_on=["Clave_Ent", "Cve. Municipio"])

In [84]:
info_country

,CLAVE,CLAVE_ENT,NOM_ENT,MUN,POB_x,NOM_MUN,CVMUN,AREAKM,hab/km2,Año,Clave_Ent,Cve. Municipio,Entidad,Municipio,Total,POB_y,Crime_Rate
0,10001,10,Durango,Canatlán,34485,Canatlán,10001,3491.96,9.875543,2019,10,10001,Durango,Canatlán,106.0,17076,620.754275
1,10002,10,Durango,Canelas,4782,Canelas,10002,892.56,5.357623,2019,10,10002,Durango,Canelas,2.0,1922,104.058273
2,10003,10,Durango,Coneto de Comonfort,4630,Coneto de Comonfort,10003,1074.47,4.309101,2019,10,10003,Durango,Coneto de Comonfort,2.0,2063,96.946195
3,10004,10,Durango,Cuencamé,37200,Cuencamé,10004,5349.36,6.954103,2019,10,10004,Durango,Cuencamé,113.0,17445,647.750072
4,10005,10,Durango,Durango,688253,Durango,10005,9285.39,74.122142,2019,10,10005,Durango,Durango,7610.0,316159,2407.016723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,9013,9,Ciudad de México,Xochimilco,419426,Xochimilco,9013,114.12,3675.306695,2019,9,9013,Ciudad de México,Xochimilco,3444.0,225339,1528.363932
2453,9014,9,Ciudad de México,Benito Juárez,430354,Benito Juárez,9014,26.70,16118.127341,2019,9,9014,Ciudad de México,Benito Juárez,11305.0,295292,3828.413909
2454,9015,9,Ciudad de México,Cuauhtémoc,547703,Cuauhtémoc,9015,32.52,16842.035670,2019,9,9015,Ciudad de México,Cuauhtémoc,17482.0,338215,5168.901438
2455,9016,9,Ciudad de México,Miguel Hidalgo,378708,Miguel Hidalgo,9016,46.39,8163.569735,2019,9,9016,Ciudad de México,Miguel Hidalgo,9345.0,241854,3863.901362


In [85]:
info_country.columns

Index(['CLAVE', 'CLAVE_ENT', 'NOM_ENT', 'MUN', 'POB_x', 'NOM_MUN', 'CVMUN',
       'AREAKM', 'hab/km2', 'Año', 'Clave_Ent', 'Cve. Municipio', 'Entidad',
       'Municipio', 'Total', 'POB_y', 'Crime_Rate'],
      dtype='object')

In [86]:
clean_df = info_country.drop(['CLAVE', 'CLAVE_ENT', 'NOM_ENT', 'MUN','NOM_MUN', 'CVMUN','Año'], axis=1)

In [87]:
clean_df

,POB_x,AREAKM,hab/km2,Clave_Ent,Cve. Municipio,Entidad,Municipio,Total,POB_y,Crime_Rate
0,34485,3491.96,9.875543,10,10001,Durango,Canatlán,106.0,17076,620.754275
1,4782,892.56,5.357623,10,10002,Durango,Canelas,2.0,1922,104.058273
2,4630,1074.47,4.309101,10,10003,Durango,Coneto de Comonfort,2.0,2063,96.946195
3,37200,5349.36,6.954103,10,10004,Durango,Cuencamé,113.0,17445,647.750072
4,688253,9285.39,74.122142,10,10005,Durango,Durango,7610.0,316159,2407.016723
...,...,...,...,...,...,...,...,...,...,...
2452,419426,114.12,3675.306695,9,9013,Ciudad de México,Xochimilco,3444.0,225339,1528.363932
2453,430354,26.70,16118.127341,9,9014,Ciudad de México,Benito Juárez,11305.0,295292,3828.413909
2454,547703,32.52,16842.035670,9,9015,Ciudad de México,Cuauhtémoc,17482.0,338215,5168.901438
2455,378708,46.39,8163.569735,9,9016,Ciudad de México,Miguel Hidalgo,9345.0,241854,3863.901362


In [88]:
clean_df.rename(columns={
    'POB_x':'Total_popul',
    'AREAKM':'Area_km2',
    'Clave_Ent':'Cve_Ent',
    'Cve. Municipio':'Cve_Mun',
    'Total':'Delitos',
    'POB_y':'Popul_Crimes',
}, inplace=True)

In [151]:
clean_df

,Total_popul,Area_km2,hab/km2,Cve_Ent,Cve_Mun,Entidad,Municipio,Delitos,Popul_Crimes,Crime_Rate
0,34485,3491.96,9.875543,10,10001,Durango,Canatlán,106.0,17076,620.754275
1,4782,892.56,5.357623,10,10002,Durango,Canelas,2.0,1922,104.058273
2,4630,1074.47,4.309101,10,10003,Durango,Coneto de Comonfort,2.0,2063,96.946195
3,37200,5349.36,6.954103,10,10004,Durango,Cuencamé,113.0,17445,647.750072
4,688253,9285.39,74.122142,10,10005,Durango,Durango,7610.0,316159,2407.016723
...,...,...,...,...,...,...,...,...,...,...
2452,419426,114.12,3675.306695,9,9013,Ciudad de México,Xochimilco,3444.0,225339,1528.363932
2453,430354,26.70,16118.127341,9,9014,Ciudad de México,Benito Juárez,11305.0,295292,3828.413909
2454,547703,32.52,16842.035670,9,9015,Ciudad de México,Cuauhtémoc,17482.0,338215,5168.901438
2455,378708,46.39,8163.569735,9,9016,Ciudad de México,Miguel Hidalgo,9345.0,241854,3863.901362


## Indicators

In [132]:
xls = pd.ExcelFile('../Indice de Desarrollo Humano Municipal 2010 2015.xlsx')

In [133]:
xls.sheet_names

['IDH municipal 2010', 'IDH municipal 2015']

In [134]:
desarrollo_df = pd.read_excel('../Indice de Desarrollo Humano Municipal 2010 2015.xlsx', sheet_name='IDH municipal 2015', skiprows=7)

In [135]:
desarrollo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 14 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Clave entidad                                2463 non-null   object 
 1   Clave municipio                              2457 non-null   float64
 2   Estado                                       2457 non-null   object 
 3   Municipio                                    2457 non-null   object 
 4   Años promedio de escolaridad                 2457 non-null   object 
 5   Años esperados de escolarización             2457 non-null   object 
 6   Ingreso per cápita anual (dólares PPC)       2457 non-null   object 
 7   Tasa de mortalidad infantil                  2457 non-null   float64
 8   Índice de educación                          2457 non-null   object 
 9   Índice de salud                              2457 non-null   float64
 10  

In [136]:
desarrollo_df.count()

Clave entidad                                  2463
Clave municipio                                2457
Estado                                         2457
Municipio                                      2457
Años promedio de escolaridad                   2457
Años esperados de escolarización               2457
Ingreso per cápita anual (dólares PPC)         2457
Tasa de mortalidad infantil                    2457
Índice de educación                            2457
Índice de salud                                2457
Índice de ingreso                              2457
Valor del Índice de Desarrollo Humano (IDH)    2457
Posición del IDH 2015                          2457
Unnamed: 13                                    2457
dtype: int64

In [137]:
desarrollo_df.isnull().sum()

Clave entidad                                  3
Clave municipio                                9
Estado                                         9
Municipio                                      9
Años promedio de escolaridad                   9
Años esperados de escolarización               9
Ingreso per cápita anual (dólares PPC)         9
Tasa de mortalidad infantil                    9
Índice de educación                            9
Índice de salud                                9
Índice de ingreso                              9
Valor del Índice de Desarrollo Humano (IDH)    9
Posición del IDH 2015                          9
Unnamed: 13                                    9
dtype: int64

In [138]:
desarrollo_df.dropna(how="any", inplace=True)

In [139]:
desarrollo_df.isna().sum()

Clave entidad                                  0
Clave municipio                                0
Estado                                         0
Municipio                                      0
Años promedio de escolaridad                   0
Años esperados de escolarización               0
Ingreso per cápita anual (dólares PPC)         0
Tasa de mortalidad infantil                    0
Índice de educación                            0
Índice de salud                                0
Índice de ingreso                              0
Valor del Índice de Desarrollo Humano (IDH)    0
Posición del IDH 2015                          0
Unnamed: 13                                    0
dtype: int64

In [140]:
desarrollo_df

,Clave entidad,Clave municipio,Estado,Municipio,Años promedio de escolaridad,Años esperados de escolarización,Ingreso per cápita anual (dólares PPC),Tasa de mortalidad infantil,Índice de educación,Índice de salud,Índice de ingreso,Valor del Índice de Desarrollo Humano (IDH),Posición del IDH 2015,Unnamed: 13
1,9,14.0,Ciudad de México,Benito Juárez,13.8318,16.2685,9544.92,5.025701,0.912964,0.971748,0.948196,0.943992,1,0
2,9,16.0,Ciudad de México,Miguel Hidalgo,12.9515,15.3655,8748.64,6.298164,0.858536,0.960939,0.935037,0.917123,2,0
3,19,19.0,Nuevo León,San Pedro Garza García,12.7893,14.9786,9748.53,11.913410,0.84238,0.913238,0.951384,0.901189,3,2
4,9,3.0,Ciudad de México,Coyoacán,12.1811,15.4159,5470.09,7.123634,0.834256,0.953926,0.864101,0.882659,4,0
5,9,15.0,Ciudad de México,Cuauhtémoc,11.9143,14.8333,6164.74,7.629359,0.809181,0.949631,0.882159,0.878448,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,8,10.0,Chihuahua,Buenaventura,ND,ND,ND,18.223442,ND,0.859634,ND,ND,ND,ND
2457,8,65.0,Chihuahua,Urique,ND,ND,ND,48.416050,ND,0.603150,ND,ND,ND,ND
2458,20,427.0,Oaxaca,Santa María Petapa,ND,ND,ND,21.057871,ND,0.835556,ND,ND,ND,ND
2459,20,140.0,Oaxaca,San Francisco Chindúa,ND,ND,ND,24.748541,ND,0.804204,ND,ND,ND,ND


In [141]:
desarrollo_df.drop("Unnamed: 13",axis=1, inplace=True)
desarrollo_df = desarrollo_df.replace("ND", np.nan )
desarrollo_df.dropna(how="any", inplace=True)

In [142]:
desarrollo_df.isnull().sum()

Clave entidad                                  0
Clave municipio                                0
Estado                                         0
Municipio                                      0
Años promedio de escolaridad                   0
Años esperados de escolarización               0
Ingreso per cápita anual (dólares PPC)         0
Tasa de mortalidad infantil                    0
Índice de educación                            0
Índice de salud                                0
Índice de ingreso                              0
Valor del Índice de Desarrollo Humano (IDH)    0
Posición del IDH 2015                          0
dtype: int64

In [143]:
desarrollo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2445 entries, 1 to 2448
Data columns (total 13 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Clave entidad                                2445 non-null   int64  
 1   Clave municipio                              2445 non-null   float64
 2   Estado                                       2445 non-null   object 
 3   Municipio                                    2445 non-null   object 
 4   Años promedio de escolaridad                 2445 non-null   float64
 5   Años esperados de escolarización             2445 non-null   float64
 6   Ingreso per cápita anual (dólares PPC)       2445 non-null   float64
 7   Tasa de mortalidad infantil                  2445 non-null   float64
 8   Índice de educación                          2445 non-null   float64
 9   Índice de salud                              2445 non-null   float64
 10  

In [144]:
desarrollo_df[["Años promedio de escolaridad", "Años esperados de escolarización", "Ingreso per cápita anual (dólares PPC)","Tasa de mortalidad infantil",
          "Índice de educación", "Índice de salud", "Índice de ingreso", "Valor del Índice de Desarrollo Humano (IDH)"]] = desarrollo_df[["Años promedio de escolaridad", "Años esperados de escolarización", "Ingreso per cápita anual (dólares PPC)","Tasa de mortalidad infantil",
          "Índice de educación", "Índice de salud", "Índice de ingreso", "Valor del Índice de Desarrollo Humano (IDH)"]].apply(pd.to_numeric,errors='coerce')

In [145]:
desarrollo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2445 entries, 1 to 2448
Data columns (total 13 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Clave entidad                                2445 non-null   int64  
 1   Clave municipio                              2445 non-null   float64
 2   Estado                                       2445 non-null   object 
 3   Municipio                                    2445 non-null   object 
 4   Años promedio de escolaridad                 2445 non-null   float64
 5   Años esperados de escolarización             2445 non-null   float64
 6   Ingreso per cápita anual (dólares PPC)       2445 non-null   float64
 7   Tasa de mortalidad infantil                  2445 non-null   float64
 8   Índice de educación                          2445 non-null   float64
 9   Índice de salud                              2445 non-null   float64
 10  

In [146]:
desarrollo_df[["Clave entidad"]] = desarrollo_df[["Clave entidad"]].astype(str)
desarrollo_df[["Clave municipio"]] = desarrollo_df[["Clave municipio"]].astype(int).astype(str)

In [148]:
desarrollo_df

,Clave entidad,Clave municipio,Estado,Municipio,Años promedio de escolaridad,Años esperados de escolarización,Ingreso per cápita anual (dólares PPC),Tasa de mortalidad infantil,Índice de educación,Índice de salud,Índice de ingreso,Valor del Índice de Desarrollo Humano (IDH),Posición del IDH 2015
1,9,14,Ciudad de México,Benito Juárez,13.831835,16.268484,9544.922852,5.025701,0.912964,0.971748,0.948196,0.943992,1.0
2,9,16,Ciudad de México,Miguel Hidalgo,12.951526,15.365477,8748.643555,6.298164,0.858536,0.960939,0.935037,0.917123,2.0
3,19,19,Nuevo León,San Pedro Garza García,12.789254,14.978569,9748.533203,11.913410,0.842380,0.913238,0.951384,0.901189,3.0
4,9,3,Ciudad de México,Coyoacán,12.181108,15.415874,5470.092773,7.123634,0.834256,0.953926,0.864101,0.882659,4.0
5,9,15,Ciudad de México,Cuauhtémoc,11.914341,14.833322,6164.735352,7.629359,0.809181,0.949631,0.882159,0.878448,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,20,16,Oaxaca,Coicoyán de las Flores,1.482081,9.526309,346.164917,45.260460,0.314022,0.629957,0.447166,0.445567,2441.0
2445,20,529,Oaxaca,Santos Reyes Yucuná,2.856512,9.568009,185.291580,41.623264,0.360995,0.660855,0.352759,0.438223,2442.0
2446,8,8,Chihuahua,Batopilas,3.305823,8.164855,599.120239,65.433395,0.336996,0.458589,0.530028,0.434292,2443.0
2447,20,242,Oaxaca,San Martín Peras,1.461217,8.763758,359.396149,51.244213,0.292145,0.579125,0.452833,0.424720,2444.0


In [111]:
desarrollo_df["Años promedio de escolaridad"] = desarrollo_df["Años promedio de escolaridad"].map("{:.3f}".format)
desarrollo_df["Años esperados de escolarización"] = desarrollo_df["Años esperados de escolarización"].map("{:.3f}".format)
desarrollo_df["Ingreso per cápita anual (dólares PPC)"] = desarrollo_df["Ingreso per cápita anual (dólares PPC)"].map("{:.3f}".format)
desarrollo_df["Tasa de mortalidad infantil"] = desarrollo_df["Tasa de mortalidad infantil"].map("{:.3f}".format)
desarrollo_df["Índice de educación"] = desarrollo_df["Índice de educación"].map("{:.3f}".format)
desarrollo_df["Índice de salud"] = desarrollo_df["Índice de salud"].map("{:.3f}".format)
desarrollo_df["Índice de ingreso"] = desarrollo_df["Índice de ingreso"].map("{:.3f}".format)
desarrollo_df["Valor del Índice de Desarrollo Humano (IDH)"] = desarrollo_df["Valor del Índice de Desarrollo Humano (IDH)"].map("{:.3f}".format)

In [150]:
desarrollo_df

,Clave entidad,Clave municipio,Estado,Municipio,Años promedio de escolaridad,Años esperados de escolarización,Ingreso per cápita anual (dólares PPC),Tasa de mortalidad infantil,Índice de educación,Índice de salud,Índice de ingreso,Valor del Índice de Desarrollo Humano (IDH),Posición del IDH 2015
1,9,14,Ciudad de México,Benito Juárez,13.831835,16.268484,9544.922852,5.025701,0.912964,0.971748,0.948196,0.943992,1.0
2,9,16,Ciudad de México,Miguel Hidalgo,12.951526,15.365477,8748.643555,6.298164,0.858536,0.960939,0.935037,0.917123,2.0
3,19,19,Nuevo León,San Pedro Garza García,12.789254,14.978569,9748.533203,11.913410,0.842380,0.913238,0.951384,0.901189,3.0
4,9,3,Ciudad de México,Coyoacán,12.181108,15.415874,5470.092773,7.123634,0.834256,0.953926,0.864101,0.882659,4.0
5,9,15,Ciudad de México,Cuauhtémoc,11.914341,14.833322,6164.735352,7.629359,0.809181,0.949631,0.882159,0.878448,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,20,16,Oaxaca,Coicoyán de las Flores,1.482081,9.526309,346.164917,45.260460,0.314022,0.629957,0.447166,0.445567,2441.0
2445,20,529,Oaxaca,Santos Reyes Yucuná,2.856512,9.568009,185.291580,41.623264,0.360995,0.660855,0.352759,0.438223,2442.0
2446,8,8,Chihuahua,Batopilas,3.305823,8.164855,599.120239,65.433395,0.336996,0.458589,0.530028,0.434292,2443.0
2447,20,242,Oaxaca,San Martín Peras,1.461217,8.763758,359.396149,51.244213,0.292145,0.579125,0.452833,0.424720,2444.0


In [154]:
indicators_df = pd.merge(desarrollo_df, clean_df, left_on=["Clave entidad", "Municipio"] ,right_on=["Cve_Ent","Municipio" ])

In [155]:
indicators_df

,Clave entidad,Clave municipio,Estado,Municipio,Años promedio de escolaridad,Años esperados de escolarización,Ingreso per cápita anual (dólares PPC),Tasa de mortalidad infantil,Índice de educación,Índice de salud,...,Posición del IDH 2015,Total_popul,Area_km2,hab/km2,Cve_Ent,Cve_Mun,Entidad,Delitos,Popul_Crimes,Crime_Rate
0,9,14,Ciudad de México,Benito Juárez,13.831835,16.268484,9544.922852,5.025701,0.912964,0.971748,...,1.0,430354,26.70,16118.127341,9,9014,Ciudad de México,11305.0,295292,3828.413909
1,9,16,Ciudad de México,Miguel Hidalgo,12.951526,15.365477,8748.643555,6.298164,0.858536,0.960939,...,2.0,378708,46.39,8163.569735,9,9016,Ciudad de México,9345.0,241854,3863.901362
2,19,19,Nuevo León,San Pedro Garza García,12.789254,14.978569,9748.533203,11.913410,0.842380,0.913238,...,3.0,136480,70.78,1928.228313,19,19019,Nuevo León,433.0,82134,527.187279
3,9,3,Ciudad de México,Coyoacán,12.181108,15.415874,5470.092773,7.123634,0.834256,0.953926,...,4.0,623683,53.92,11566.821217,9,9003,Ciudad de México,7896.0,392362,2012.427299
4,9,15,Ciudad de México,Cuauhtémoc,11.914341,14.833322,6164.735352,7.629359,0.809181,0.949631,...,5.0,547703,32.52,16842.035670,9,9015,Ciudad de México,17482.0,338215,5168.901438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,20,352,Oaxaca,San Simón Zahuatlán,2.179156,8.293554,333.424408,40.802273,0.303015,0.667829,...,2440.0,4323,46.09,93.794749,20,20352,Oaxaca,5.0,1874,266.808965
2414,20,16,Oaxaca,Coicoyán de las Flores,1.482081,9.526309,346.164917,45.260460,0.314022,0.629957,...,2441.0,10243,126.09,81.235625,20,20016,Oaxaca,6.0,4641,129.282482
2415,20,529,Oaxaca,Santos Reyes Yucuná,2.856512,9.568009,185.291580,41.623264,0.360995,0.660855,...,2442.0,1353,60.90,22.216749,20,20529,Oaxaca,0.0,602,0.000000
2416,20,242,Oaxaca,San Martín Peras,1.461217,8.763758,359.396149,51.244213,0.292145,0.579125,...,2444.0,12529,242.70,51.623403,20,20242,Oaxaca,6.0,5464,109.809663


In [156]:
indicators_df.columns

Index(['Clave entidad', 'Clave municipio', 'Estado', 'Municipio',
       'Años promedio de escolaridad', 'Años esperados de escolarización',
       'Ingreso per cápita anual (dólares PPC)', 'Tasa de mortalidad infantil',
       'Índice de educación', 'Índice de salud', 'Índice de ingreso',
       'Valor del Índice de Desarrollo Humano (IDH)', 'Posición del IDH 2015',
       'Total_popul', 'Area_km2', 'hab/km2', 'Cve_Ent', 'Cve_Mun', 'Entidad',
       'Delitos', 'Popul_Crimes', 'Crime_Rate'],
      dtype='object')

In [158]:
clean_indicators_df = indicators_df.drop(['Años esperados de escolarización','Años promedio de escolaridad',
                                         'Tasa de mortalidad infantil','Cve_Ent', 'Cve_Mun', 'Entidad'], axis=1)

In [160]:
clean_indicators_df.head(50)

,Clave entidad,Clave municipio,Estado,Municipio,Ingreso per cápita anual (dólares PPC),Índice de educación,Índice de salud,Índice de ingreso,Valor del Índice de Desarrollo Humano (IDH),Posición del IDH 2015,Total_popul,Area_km2,hab/km2,Delitos,Popul_Crimes,Crime_Rate
0,9,14,Ciudad de México,Benito Juárez,9544.922852,0.912964,0.971748,0.948196,0.943992,1.0,430354,26.70,16118.127341,11305.0,295292,3828.413909
1,9,16,Ciudad de México,Miguel Hidalgo,8748.643555,0.858536,0.960939,0.935037,0.917123,2.0,378708,46.39,8163.569735,9345.0,241854,3863.901362
2,19,19,Nuevo León,San Pedro Garza García,9748.533203,0.842380,0.913238,0.951384,0.901189,3.0,136480,70.78,1928.228313,433.0,82134,527.187279
3,9,3,Ciudad de México,Coyoacán,5470.092773,0.834256,0.953926,0.864101,0.882659,4.0,623683,53.92,11566.821217,7896.0,392362,2012.427299
4,9,15,Ciudad de México,Cuauhtémoc,6164.735352,0.809181,0.949631,0.882159,0.878448,5.0,547703,32.52,16842.035670,17482.0,338215,5168.901438
5,20,350,Oaxaca,San Sebastián Tutla,4259.788574,0.839797,0.942688,0.826325,0.868089,6.0,19306,7.33,2633.833561,130.0,13779,943.464693
6,22,6,Querétaro,Corregidora,5855.110352,0.780925,0.952736,0.874375,0.866483,7.0,202268,234.89,861.117970,2336.0,135152,1728.424293
7,9,6,Ciudad de México,Iztacalco,5553.652832,0.781534,0.943205,0.866391,0.861171,8.0,394448,23.10,17075.670996,5309.0,238260,2228.238059
8,19,46,Nuevo León,San Nicolás de los Garza,5495.046875,0.778772,0.940475,0.864788,0.858794,9.0,473285,60.17,7865.796909,1615.0,290066,556.769839
9,9,2,Ciudad de México,Azcapotzalco,4682.480957,0.785470,0.943627,0.840616,0.854101,10.0,409860,33.52,12227.326969,5595.0,253977,2202.955386


In [161]:
clean_indicators_df["Ingreso per cápita anual (dólares PPC)"] = clean_indicators_df["Ingreso per cápita anual (dólares PPC)"].map("{:.2f}".format)
clean_indicators_df["Índice de educación"] = clean_indicators_df["Índice de educación"].map("{:.2f}".format)
clean_indicators_df["Índice de salud"] = clean_indicators_df["Índice de salud"].map("{:.2f}".format)
clean_indicators_df["Índice de ingreso"] = clean_indicators_df["Índice de ingreso"].map("{:.2f}".format)
clean_indicators_df["Valor del Índice de Desarrollo Humano (IDH)"] = clean_indicators_df["Valor del Índice de Desarrollo Humano (IDH)"].map("{:.2f}".format)
clean_indicators_df["hab/km2"] = clean_indicators_df["hab/km2"].map("{:.2f}".format)
clean_indicators_df["Crime_Rate"] = clean_indicators_df["Crime_Rate"].map("{:.2f}".format)

In [162]:
clean_indicators_df

,Clave entidad,Clave municipio,Estado,Municipio,Ingreso per cápita anual (dólares PPC),Índice de educación,Índice de salud,Índice de ingreso,Valor del Índice de Desarrollo Humano (IDH),Posición del IDH 2015,Total_popul,Area_km2,hab/km2,Delitos,Popul_Crimes,Crime_Rate
0,9,14,Ciudad de México,Benito Juárez,9544.92,0.91,0.97,0.95,0.94,1.0,430354,26.70,16118.13,11305.0,295292,3828.41
1,9,16,Ciudad de México,Miguel Hidalgo,8748.64,0.86,0.96,0.94,0.92,2.0,378708,46.39,8163.57,9345.0,241854,3863.90
2,19,19,Nuevo León,San Pedro Garza García,9748.53,0.84,0.91,0.95,0.90,3.0,136480,70.78,1928.23,433.0,82134,527.19
3,9,3,Ciudad de México,Coyoacán,5470.09,0.83,0.95,0.86,0.88,4.0,623683,53.92,11566.82,7896.0,392362,2012.43
4,9,15,Ciudad de México,Cuauhtémoc,6164.74,0.81,0.95,0.88,0.88,5.0,547703,32.52,16842.04,17482.0,338215,5168.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,20,352,Oaxaca,San Simón Zahuatlán,333.42,0.30,0.67,0.44,0.45,2440.0,4323,46.09,93.79,5.0,1874,266.81
2414,20,16,Oaxaca,Coicoyán de las Flores,346.16,0.31,0.63,0.45,0.45,2441.0,10243,126.09,81.24,6.0,4641,129.28
2415,20,529,Oaxaca,Santos Reyes Yucuná,185.29,0.36,0.66,0.35,0.44,2442.0,1353,60.90,22.22,0.0,602,0.00
2416,20,242,Oaxaca,San Martín Peras,359.40,0.29,0.58,0.45,0.42,2444.0,12529,242.70,51.62,6.0,5464,109.81


In [163]:
clean_indicators_df.to_csv("indicators_Mexico.csv")